# 1. Imports

In [1]:
import json
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, confusion_matrix, adjusted_rand_score
from sklearn.svm import SVC
import pickle
from pandas import DataFrame
from sklearn.linear_model import RidgeClassifierCV
from sklearn.pipeline import Pipeline
from sktime.transformations.panel.rocket import Rocket
from collections import Counter
from matplotlib import pyplot
from numpy import where
import statistics
from sklearn.model_selection import StratifiedKFold
from sktime.transformations.panel.rocket import MiniRocket, MiniRocketMultivariate
import matplotlib.pyplot as plt
from datetime import datetime

# 2. Data loading

### 2.1 Original Data

In [25]:
#time
now = datetime.now()
print("EXPERIMENTS START TIME =", now)

EXPERIMENTS START TIME = 2022-11-30 23:29:26.947922


In [26]:
p1_data = pd.read_pickle(r'../og_data/partition1_data.pkl')
p2_data = pd.read_pickle(r'../og_data/partition2_data.pkl')
p3_data = pd.read_pickle(r'../og_data/partition3_data.pkl')
p4_data = pd.read_pickle(r'../og_data/partition4_data.pkl')
p5_data = pd.read_pickle(r'../og_data/partition5_data.pkl')
p1_labels = pd.read_pickle(r'../og_data/partition1_labels.pkl')
p2_labels = pd.read_pickle(r'../og_data/partition2_labels.pkl')
p3_labels = pd.read_pickle(r'../og_data/partition3_labels.pkl')
p4_labels = pd.read_pickle(r'../og_data/partition4_labels.pkl')
p5_labels = pd.read_pickle(r'../og_data/partition5_labels.pkl')

### 2.2 Converting labels into Binary

In [27]:
# converting labels into binary class
p1_labels_binary = np.where(p1_labels == 'B', 'NF', p1_labels)
p1_labels_binary = np.where(p1_labels_binary == 'C', 'NF', p1_labels_binary)
p1_labels_binary = np.where(p1_labels_binary == 'F', 'NF', p1_labels_binary)
p1_labels_binary = np.where(p1_labels_binary == 'M', 'F', p1_labels_binary)
p1_labels_binary = np.where(p1_labels_binary == 'X', 'F', p1_labels_binary)

p2_labels_binary = np.where(p2_labels == 'B', 'NF', p2_labels)
p2_labels_binary = np.where(p2_labels_binary == 'C', 'NF', p2_labels_binary)
p2_labels_binary = np.where(p2_labels_binary == 'F', 'NF', p2_labels_binary)
p2_labels_binary = np.where(p2_labels_binary == 'M', 'F', p2_labels_binary)
p2_labels_binary = np.where(p2_labels_binary == 'X', 'F', p2_labels_binary)

p3_labels_binary = np.where(p3_labels == 'B', 'NF', p3_labels)
p3_labels_binary = np.where(p3_labels_binary == 'C', 'NF', p3_labels_binary)
p3_labels_binary = np.where(p3_labels_binary == 'F', 'NF', p3_labels_binary)
p3_labels_binary = np.where(p3_labels_binary == 'M', 'F', p3_labels_binary)
p3_labels_binary = np.where(p3_labels_binary == 'X', 'F', p3_labels_binary)

p4_labels_binary = np.where(p4_labels == 'B', 'NF', p4_labels)
p4_labels_binary = np.where(p4_labels_binary == 'C', 'NF', p4_labels_binary)
p4_labels_binary = np.where(p4_labels_binary == 'F', 'NF', p4_labels_binary)
p4_labels_binary = np.where(p4_labels_binary == 'M', 'F', p4_labels_binary)
p4_labels_binary = np.where(p4_labels_binary == 'X', 'F', p4_labels_binary)

p5_labels_binary = np.where(p5_labels == 'B', 'NF', p5_labels)
p5_labels_binary = np.where(p5_labels_binary == 'C', 'NF', p5_labels_binary)
p5_labels_binary = np.where(p5_labels_binary == 'F', 'NF', p5_labels_binary)
p5_labels_binary = np.where(p5_labels_binary == 'M', 'F', p5_labels_binary)
p5_labels_binary = np.where(p5_labels_binary == 'X', 'F', p5_labels_binary)

# 3. All possible partition pair

Binary Labels

In [28]:
pp_20_bin = [ 
[p1_data, p2_data, p1_labels_binary, p2_labels_binary],

[p1_data, p3_data, p1_labels_binary, p3_labels_binary],

[p1_data, p4_data, p1_labels_binary, p4_labels_binary],

[p1_data, p5_data, p1_labels_binary, p5_labels_binary],

[p2_data, p1_data, p2_labels_binary, p1_labels_binary],

[p2_data, p3_data, p2_labels_binary, p3_labels_binary],

[p2_data, p4_data, p2_labels_binary, p4_labels_binary],

[p2_data, p5_data, p2_labels_binary, p5_labels_binary],

[p3_data, p1_data, p3_labels_binary, p1_labels_binary],

[p3_data, p2_data, p3_labels_binary, p2_labels_binary],

[p3_data, p4_data, p3_labels_binary, p4_labels_binary],

[p3_data, p5_data, p3_labels_binary, p5_labels_binary],

[p4_data, p1_data, p4_labels_binary, p1_labels_binary],

[p4_data, p2_data, p4_labels_binary, p2_labels_binary],

[p4_data, p3_data, p4_labels_binary, p3_labels_binary],

[p4_data, p5_data, p4_labels_binary, p5_labels_binary],

[p5_data, p1_data, p5_labels_binary, p1_labels_binary],

[p5_data, p2_data, p5_labels_binary, p2_labels_binary],

[p5_data, p3_data, p5_labels_binary, p3_labels_binary],

[p5_data, p4_data, p5_labels_binary, p4_labels_binary],
]

All Labels

In [29]:
pp_20_all = [ 
[p1_data, p2_data, p1_labels, p2_labels],

[p1_data, p3_data, p1_labels, p3_labels],

[p1_data, p4_data, p1_labels, p4_labels],

[p1_data, p5_data, p1_labels, p5_labels],

[p2_data, p1_data, p2_labels, p1_labels],

[p2_data, p3_data, p2_labels, p3_labels],

[p2_data, p4_data, p2_labels, p4_labels],

[p2_data, p5_data, p2_labels, p5_labels],

[p3_data, p1_data, p3_labels, p1_labels],

[p3_data, p2_data, p3_labels, p2_labels],

[p3_data, p4_data, p3_labels, p4_labels],

[p3_data, p5_data, p3_labels, p5_labels],

[p4_data, p1_data, p4_labels, p1_labels],

[p4_data, p2_data, p4_labels, p2_labels],

[p4_data, p3_data, p4_labels, p3_labels],

[p4_data, p5_data, p4_labels, p5_labels],

[p5_data, p1_data, p5_labels, p1_labels],

[p5_data, p2_data, p5_labels, p2_labels],

[p5_data, p3_data, p5_labels, p3_labels],

[p5_data, p4_data, p5_labels, p4_labels],
]

# 4. Experiment

In [30]:
version1 = MiniRocketMultivariate()
version2 = Rocket()

In [32]:
def train_model_bin(version):

    cm = []
    
    for i in pp_20_bin:

        version.fit(i[0])
        X_train_transform = version.transform(i[0])

        classifier = RidgeClassifierCV(alphas=np.logspace(-3, 3, 10), normalize=True)
        classifier.fit(X_train_transform, i[2])

        X_test_transform = version.transform(i[1])

        y_pred = classifier.predict(X_test_transform)

        #confusion matrix
        score = confusion_matrix(i[3],y_pred)
        cm.append(score)

    tss = []
    hss2 = []
    
    for i in cm:
        
        TN, FP, FN, TP = i.ravel()
        TN = np.array(TN)
        FP = np.array(FP)
        FN = np.array(FN)
        TP = np.array(TP)
        
        acc = (TP + TN) / (TP + FN + TN + FP)
        pr_pos = TP/(TP + FP)
        pr_neg = TN/(TN + FN)
        rc_pos = TP/(TP + FN)
        rc_neg = TN/(TN + FP)
        f1_pos = (2 * pr_pos * rc_pos) / (pr_pos + rc_pos)
        f1_neg = (2 * pr_neg * rc_neg) / (pr_neg + rc_neg)
        
        P = TP + FN
        N = TN + FP
        
        HSS1 = (TP + TN - N) / P
        HSS2 = (2*((TP*TN)-(FP*FN)))/(P*(FN+TN)+(TP+FP)*N)
        
        CH = ((TP+FP)*(TP+FN))/(P+N)
        GS = (TP-CH)/(TP+FP+FN-CH)
        
        TSS = ((TP*TN)-(FP*FN))/(P*N)
        
        tss.append(TSS)
        hss2.append(HSS2)
        
    return tss, hss2

def train_model_all(version):

    cm = []
    
    for i in pp_20_all:

        version.fit(i[0])
        X_train_transform = version.transform(i[0])

        classifier = RidgeClassifierCV(alphas=np.logspace(-3, 3, 10), normalize=True)
        classifier.fit(X_train_transform, i[2])

        X_test_transform = version.transform(i[1])

        y_pred = classifier.predict(X_test_transform)

        #confusion matrix
        score = confusion_matrix(i[3],y_pred)
        cm.append(score)

    tss = []
    hss2 = []
    
    for i in cm:
        
        listt = list(i.ravel())
        TP = listt[18] + listt[19] + listt[23] + listt[24]
        FN = listt[15] + listt[16] + listt[17] + listt[20] + listt[21] + listt[22]
        FP = listt[3] + listt[4] + listt[8] + listt[9] + listt[13] + listt[14]
        TN = listt[0] + listt[1] + listt[2] + listt[5] + listt[6] + listt[7] + listt[10] + listt[11] + listt[12]

        acc = (TP + TN) / (TP + FN + TN + FP)
        pr_pos = TP/(TP + FP)
        pr_neg = TN/(TN + FN)
        rc_pos = TP/(TP + FN)
        rc_neg = TN/(TN + FP)
        f1_pos = (2 * pr_pos * rc_pos) / (pr_pos + rc_pos)
        f1_neg = (2 * pr_neg * rc_neg) / (pr_neg + rc_neg)
        
        P = TP + FN
        N = TN + FP
        
        HSS1 = (TP + TN - N) / P
        HSS2 = (2*((TP*TN)-(FP*FN)))/(P*(FN+TN)+(TP+FP)*N)
        
        CH = ((TP+FP)*(TP+FN))/(P+N)
        GS = (TP-CH)/(TP+FP+FN-CH)
        
        TSS = ((TP*TN)-(FP*FN))/(P*N)
        
        tss.append(TSS)
        hss2.append(HSS2)
        
    return tss, hss2

### 4.1 Binary Labels

#### 4.1.1 Mini Rocket

In [33]:
#time
now = datetime.now()
print("MiniROCKET with Binary Labels START TIME =", now)

MiniROCKET with Binary Labels START TIME = 2022-11-30 23:30:57.763588


In [9]:
tss_mr_bin, hss2_mr_bin = train_model_bin(version1)
print(tss_mr_bin)
print(hss2_mr_bin)

#### 4.1.2 Rocket

In [20]:
#time
now = datetime.now()
print("ROCKET with Binary Labels START TIME =", now)

ROCKET with Binary Labels START TIME = 2022-11-30 23:25:14.260174


In [ ]:
tss_r_bin, hss2_r_bin = train_model_bin(version2)
print(tss_r_bin)
print(hss2_r_bin)

### 4.2 All Labels

#### 4.2.1 Mini Rocket

In [21]:
#time
now = datetime.now()
print("MiniROCKET with All Labels START TIME =", now)

MiniROCKET with All Labels START TIME = 2022-11-30 23:26:33.547899


In [11]:
tss_mr_all, hss2_mr_all = train_model_all(version1)
print(tss_mr_all)
print(hss2_mr_all)

#### 4.2.2 ROCKET

In [23]:
#time
now = datetime.now()
print("ROCKET with All Labels START TIME =", now)

ROCKET with All Labels START TIME = 2022-11-30 23:28:42.921451


In [13]:
tss_r_all, hss2_r_all = train_model(version3)
print(tss_r_all)
print(hss2_r_all)

In [34]:
#time
now = datetime.now()
print("EXPERIMENTS FINISHED TIME =", now)

EXPERIMENTS FINISHED TIME = 2022-11-30 23:31:29.100895
